In [ ]:
!pip install -q -u trl transformers accelerate git+git://github.com/huggingface/peft.git
!pip install -q -U bitsandbytes einops wandB
!pip install trl
!pip install peft
!pip  install fitz
!pip install frontend
!pip install PyMuPDF
!pip install datasets
!pip install optim


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install optimum
!pip install auto-gptq
!pip install optimum auto-gptq
!pip show transformers optimum auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.7/453.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 48.1 MB/s eta 0:00:00
Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: auto_gptq, optimum, peft, trl
---
Name: optim

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
from google.colab import drive
import fitz
import os

In [ ]:
drive.mount('/content/drive')
drive_path ='/content/drive/MyDrive/Baymax/'

Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/MyDrive/Baymax/Dataset_model/"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf:
        for page_num in range(len(pdf)):
            page = pdf.load_page(page_num)
            text += page.get_text()
    return text

In [ ]:
books = ['MedBook1.pdf', 'MedBook2.pdf', 'MedBook3.pdf', 'MedBook4.pdf', 'MedBook5.pdf', 'MedBook6.pdf', 'MedBook7.pdf']
all_texts = []
for book in books:
  book_path = os.path.join(dataset_path, book)
  text = extract_text_from_pdf(book_path)
  all_texts.append(text)

In [ ]:
from datasets import Dataset
data  = {'text': all_texts}
dataset = Dataset.from_dict(data)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

question = "What are the symptoms of COVID-19?"

inputs = tokenizer.encode(question, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Question: {question}")
print(f"Answer: {answer}")

while True:
    query = input("Ask a medical question (or 'exit' to stop): ")
    if query.lower() == "exit":
        break
    inputs = tokenizer(query, return_tensors="pt").to(device)
    outputs = model.generate(inputs['input_ids'], max_length=300, top_p=0.9, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Chatbot: {response}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: What are the symptoms of COVID-19?
Answer: What are the symptoms of COVID-19?

COVID is a common, but not fatal, symptom of chronic obstructive pulmonary disease (COPD). It is caused by a combination of a number of factors, including:
.
 (1) The presence of an obstruction. (2) A lack of oxygen. The condition is usually caused when the airway is blocked by obstructions. It can be caused either by: (a) a lack or lack't breathing.
Ask a medical question (or 'exit' to stop): exit


In [ ]:
pip install transformers datasets torch

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True, truncation=True, padding="max_length", max_length=512)


In [ ]:
from transformers import Trainer, TrainingArguments

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")  # Assuming you have labels in your inputs
        if labels is None:
            print("Labels are missing from the inputs!")  # Add this line for debugging
        outputs = model(**inputs)
        logits = outputs.get("logits")  # Get the logits from the model's output
        if logits is None:
            print("Logits are missing from the model output!")  # Add this line

        # Check if either logits or labels is None and return a dummy loss if so
        if logits is None or labels is None:
            print("Returning dummy loss")
            return torch.tensor(0.0, requires_grad=True, device=model.device) # Return a dummy loss with requires_grad=True and move it to the model's device

        loss_fn = torch.nn.CrossEntropyLoss()  # Example loss function - choose appropriate one
        loss = loss_fn(logits.view(-1, self.model.config.vocab_size), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
tokenizer.pad_token = tokenizer.eos_token # Set the padding token to the end-of-sequence token
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"), batched=True)

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save results
    overwrite_output_dir=True,       # Overwrite content if exists
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=4,   # Batch size per GPU/CPU
    per_device_eval_batch_size=4,    # Evaluation batch size
    warmup_steps=500,                # Warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=10,
)


In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Labels are missing from the inputs!
Returning dummy loss


Step,Training Loss


Labels are missing from the inputs!
Returning dummy loss
Labels are missing from the inputs!
Returning dummy loss
Labels are missing from the inputs!
Returning dummy loss
Labels are missing from the inputs!
Returning dummy loss
Labels are missing from the inputs!
Returning dummy loss


TrainOutput(global_step=6, training_loss=0.0, metrics={'train_runtime': 26.3638, 'train_samples_per_second': 0.797, 'train_steps_per_second': 0.228, 'total_flos': 10974265344000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
while True:
    query = input("Ask a medical question (or 'exit' to stop): ")
    if query.lower() == "exit":
        break
    # Tokenize and generate the response
    inputs = tokenizer(query, return_tensors="pt").to(device)
    # Generate a response from the fine-tuned model
    outputs = model.generate(inputs['input_ids'], max_length=300, top_p=0.9, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the chatbot's response
    print(f"Chatbot: {response}")

Ask a medical question (or 'exit' to stop): what are the symptoms to know that the virus attacked humon bodies?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: what are the symptoms to know that the virus attacked humon bodies? That's why I got to get to this place and take this one question that's on this site and we are very interested in this: What does that mean and does it matter? I have to come across the same idea, but here we are, you have to keep on the edge and we are going to keep this down and get the virus out. It's important that we get this to be our next issue. We are going to get through the first issues about that. So, yeah. You keep the edge and I get to this question, how did you get to this point of the virus. I mean, this was a big question. I went in this, I said, that it's important to know that this is where we are. You know I mean you know we know you were coming. You know you knew we got there with this. You know you get to a point where you can see what the virus is and then get there. You get there and get to this point, it's when we get to the point of the virus and I mean to take the whole thing in its 

In [ ]:
import pickle
pickle_path = '/content/drive/MyDrive/final_model.pkl'

In [ ]:
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)


In [ ]:
with open('final_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)